In [1]:
from pyscf import gto, scf, ao2mo, fci
from GeneralisedWick import *
from scipy.optimize import fsolve

In [2]:
def commutator(operator1, operator2):
    return operator1 * operator2 + (-1) * operator2 * operator1

In [3]:
def BCHSimilarityTransform(H, T, order):
    result = H
    for k in range(order):
        nestedCommutator = H
        for i in range(k + 1):
            nestedCommutator = commutator(nestedCommutator, T)
        result += (1 / factorial(k + 1)) * nestedCommutator
    return result

In [4]:
def projectionManifold(excitationLevel):
    return Tensor('\Phi', ['h'] * excitationLevel, ['p'] * excitationLevel)

In [5]:
h1Tensor = Tensor("f", ['g'], ['g'])
h2Tensor = Tensor("v", ['g', 'g'], ['g', 'g'])
t2Tensor = Tensor("{t_{2}}", ['p', 'p'], ['h', 'h'])

In [12]:
bohr = 0.529177249

H2sep = 1.605 * bohr

mol = gto.Mole()
mol.verbose = 1
mol.atom = 'H 0 0 0; H 0 0 ' + str(H2sep)
mol.basis = 'sto-3g'
mol.spin = 0
mol.build()

Enuc = mol.energy_nuc()

mf = scf.ROHF(mol)
mf.kernel()

cisolver = fci.FCI(mol, mf.mo_coeff)

Norbs = mol.nao
Nocc = mf.nelectron_alpha
vacuum = [1] * Nocc + [0] * (Norbs - Nocc)

h1 = mf.mo_coeff.T.dot(mf.get_hcore()).dot(mf.mo_coeff)
eri = ao2mo.kernel(mol, mf.mo_coeff, compact=False)

In [289]:
from pyscf import cc, ccn

In [294]:
trueCCD = cc.CCSD(mf).kernel()

In [311]:
trueCCD

(-0.02581552431935216,
 (array([[-1.61881792e-16]]), array([[-6.76294976e-17]])),
 (array([[[[0.]]]]), array([[[[-0.13766666]]]]), array([[[[0.]]]])))

In [298]:
trueAmplitudeArray = trueCCD[2][1]

In [299]:
trueAmplitudeArray

array([[[[-0.13766666]]]])

In [300]:
t2Tensor.array

array([[[[-0.13766666]]]])

In [13]:
def get2bodyHamiltonianArray(mf_):
    eri = ao2mo.kernel(mf_.mol, mf_.mo_coeff)
    Norbs_ = mf_.mol.nao
    hamiltonian2BodyArray = np.zeros((Norbs_,Norbs_,Norbs_,Norbs_))
    for p in range(Norbs_):
        for q in range(Norbs_):
            for r in range(p + 1):
                for s in range(q + 1):
                    x = int(p + Norbs_ * r - 0.5 * r * (r + 1))
                    y = int(q + Norbs_ * s - 0.5 * s * (s + 1))
                    if p == r and q == s:
                        hamiltonian2BodyArray[p,q,r,s] += eri[x, y]
                    else:
                        hamiltonian2BodyArray[p,q,r,s] += eri[x, y]
                        hamiltonian2BodyArray[r,s,p,q] += np.conjugate(eri[x, y])
    return hamiltonian2BodyArray

In [14]:
h1Tensor.array = h1
h2Tensor.array = eri.reshape((Norbs, Norbs, Norbs, Norbs)).swapaxes(2,3).swapaxes(1,2)
t2Tensor.getShape(vacuum)

In [313]:
#t2Tensor.array[0, 0, 0, 0] = -0.275333314977893 * 0.5
t2Tensor.array = trueAmplitudeArray

In [318]:
h1

array([[-1.18985062e+00,  2.60021255e-17],
       [-4.71423569e-17, -5.33749102e-01]])

In [317]:
eri.reshape((Norbs, Norbs, Norbs, Norbs)).swapaxes(2,3).swapaxes(1,2)

array([[[[6.54009511e-01, 8.32667268e-17],
         [1.17961196e-16, 1.87521981e-01]],

        [[1.11022302e-16, 6.45249427e-01],
         [1.87521981e-01, 1.11022302e-16]]],


       [[[1.04083409e-16, 1.87521981e-01],
         [6.45249427e-01, 1.38777878e-16]],

        [[1.87521981e-01, 5.55111512e-17],
         [2.22044605e-16, 6.78136184e-01]]]])

Hard code spin-free equations from Bartlett & Shavitt Chapter 12

In [26]:
h1Diagrams = h1Tensor.getAllDiagrams(vacuum)
h2Diagrams = h2Tensor.getAllDiagrams(vacuum)

In [33]:
print(*h1Diagrams)

f_{h}^{h} f_{h}^{p} f_{p}^{h} f_{p}^{p}


In [36]:
print(*h2Diagrams)

v_{hh}^{hh} v_{hh}^{hp} v_{hh}^{ph} v_{hh}^{pp} v_{hp}^{hh} v_{hp}^{hp} v_{hp}^{ph} v_{hp}^{pp} v_{ph}^{hh} v_{ph}^{hp} v_{ph}^{ph} v_{ph}^{pp} v_{pp}^{hh} v_{pp}^{hp} v_{pp}^{ph} v_{pp}^{pp}


In [38]:
I = Index("I", True)
J = Index("J", True)
K = Index("K", True)
L = Index("L", True)
A = Index("A", False)
B = Index("B", False)
C = Index("C", False)
D = Index("D", False)

In [104]:
D1 = TensorSum([TensorProduct([h2Diagrams[12]], vertexList=[Vertex(h2Diagrams[12], [A,B], [I,J])])])

In [105]:
D2a = TensorSum([TensorProduct([h1Diagrams[3], t2Tensor], vertexList=[Vertex(h1Diagrams[3], [B], [C]), Vertex(t2Tensor, [A,C], [I,J])]), TensorProduct([h1Diagrams[3], t2Tensor], vertexList=[Vertex(h1Diagrams[3], [A], [C]), Vertex(t2Tensor, [B,C], [J,I])])])

In [106]:
D2b = TensorSum([TensorProduct([h1Diagrams[0], t2Tensor], vertexList=[Vertex(h1Diagrams[0], [K], [J]), Vertex(t2Tensor, [A,B], [I,K])]), TensorProduct([h1Diagrams[0], t2Tensor], vertexList=[Vertex(h1Diagrams[0], [K], [I]), Vertex(t2Tensor, [B,A], [J,K])])])

In [107]:
D2c = TensorSum([TensorProduct([h2Diagrams[15], t2Tensor], vertexList=[Vertex(h2Diagrams[15], [A,B], [C,D]), Vertex(t2Tensor, [C,D], [I,J])]), TensorProduct([h2Diagrams[15], t2Tensor], vertexList=[Vertex(h2Diagrams[15], [B,A], [C,D]), Vertex(t2Tensor, [C,D], [J,I])])])

In [108]:
D2d = TensorSum([TensorProduct([h2Diagrams[0], t2Tensor], vertexList=[Vertex(h2Diagrams[0], [K,L], [I,J]), Vertex(t2Tensor, [A,B], [K,L])]), TensorProduct([h2Diagrams[0], t2Tensor], vertexList=[Vertex(h2Diagrams[0], [K,L], [J,I]), Vertex(t2Tensor, [B,A], [K,L])])])

In [109]:
D2e = TensorSum([TensorProduct([h2Diagrams[6], t2Tensor], vertexList=[Vertex(h2Diagrams[6], [K,B], [C,J]), Vertex(t2Tensor, [A,C], [I,K])]), TensorProduct([h2Diagrams[6], t2Tensor], vertexList=[Vertex(h2Diagrams[6], [K,A], [C,I]), Vertex(t2Tensor, [B,C], [J,K])])])

In [110]:
D2ex1 = TensorSum([TensorProduct([h2Diagrams[6], t2Tensor], vertexList=[Vertex(h2Diagrams[6], [K,B], [C,J]), Vertex(t2Tensor, [A,C], [K,I])]), TensorProduct([h2Diagrams[6], t2Tensor], vertexList=[Vertex(h2Diagrams[6], [K,A], [C,I]), Vertex(t2Tensor, [B,C], [K,J])])])

In [111]:
D2ex2 = TensorSum([TensorProduct([h2Diagrams[5], t2Tensor], vertexList=[Vertex(h2Diagrams[5], [K,B], [I,C]), Vertex(t2Tensor, [A,C], [K,J])]), TensorProduct([h2Diagrams[5], t2Tensor], vertexList=[Vertex(h2Diagrams[5], [K,A], [J,C]), Vertex(t2Tensor, [B,C], [K,I])])])

In [112]:
D2ex3 = TensorSum([TensorProduct([h2Diagrams[5], t2Tensor], vertexList=[Vertex(h2Diagrams[5], [K,B], [J,C]), Vertex(t2Tensor, [A,C], [I,K])]), TensorProduct([h2Diagrams[5], t2Tensor], vertexList=[Vertex(h2Diagrams[5], [K,A], [I,C]), Vertex(t2Tensor, [B,C], [J,K])])])

In [113]:
D3a = TensorSum([TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [K,L], [C,D]), Vertex(t2Tensor, [C,D], [I,J]), Vertex(t2Tensor, [A,B], [K,L])]), TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [K,L], [C,D]), Vertex(t2Tensor, [C,D], [J,I]), Vertex(t2Tensor, [B,A], [K,L])])])

In [114]:
D3b = TensorSum([TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [K,L], [C,D]), Vertex(t2Tensor, [A,C], [I,K]), Vertex(t2Tensor, [D,B], [L,J])]), TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [K,L], [C,D]), Vertex(t2Tensor, [B,C], [J,K]), Vertex(t2Tensor, [D,A], [L,I])])])

In [115]:
D3bx1 = TensorSum([TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [K,L], [C,D]), Vertex(t2Tensor, [A,C], [I,K]), Vertex(t2Tensor, [D,B], [J,L])]), TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [K,L], [C,D]), Vertex(t2Tensor, [B,C], [J,K]), Vertex(t2Tensor, [D,A], [I,L])])])

In [116]:
D3bx2 = TensorSum([TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [K,L], [C,D]), Vertex(t2Tensor, [C,A], [I,K]), Vertex(t2Tensor, [B,D], [L,J])]), TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [K,L], [C,D]), Vertex(t2Tensor, [C,B], [J,K]), Vertex(t2Tensor, [A,D], [L,I])])])

In [117]:
D3bx3 = TensorSum([TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [K,L], [C,D]), Vertex(t2Tensor, [A,D], [I,K]), Vertex(t2Tensor, [C,B], [L,J])]), TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [K,L], [C,D]), Vertex(t2Tensor, [B,D], [J,K]), Vertex(t2Tensor, [C,A], [L,I])])])

In [118]:
D3bx4 = TensorSum([TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [K,L], [C,D]), Vertex(t2Tensor, [A,D], [K,I]), Vertex(t2Tensor, [C,B], [L,J])]), TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [K,L], [C,D]), Vertex(t2Tensor, [B,D], [K,J]), Vertex(t2Tensor, [C,A], [L,I])])])

In [119]:
D3bx5 = TensorSum([TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [K,L], [C,D]), Vertex(t2Tensor, [C,B], [I,L]), Vertex(t2Tensor, [A,D], [K,J])]), TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [K,L], [C,D]), Vertex(t2Tensor, [C,A], [J,L]), Vertex(t2Tensor, [B,D], [K,I])])])

In [120]:
D3c = TensorSum([TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [K,L], [C,D]), Vertex(t2Tensor, [C,D], [K,I]), Vertex(t2Tensor, [A,B], [L,J])]), TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [K,L], [C,D]), Vertex(t2Tensor, [C,D], [K,J]), Vertex(t2Tensor, [B,A], [L,I])])])

In [121]:
D3cx = TensorSum([TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [K,L], [C,D]), Vertex(t2Tensor, [C,D], [I,K]), Vertex(t2Tensor, [A,B], [L,J])]), TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [K,L], [C,D]), Vertex(t2Tensor, [C,D], [J,K]), Vertex(t2Tensor, [B,A], [L,I])])])

In [122]:
D3d = TensorSum([TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [K,L], [C,D]), Vertex(t2Tensor, [C,A], [K,L]), Vertex(t2Tensor, [D,B], [I,J])]), TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [K,L], [C,D]), Vertex(t2Tensor, [C,B], [K,L]), Vertex(t2Tensor, [D,A], [J,I])])])

In [123]:
D3dx = TensorSum([TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [K,L], [C,D]), Vertex(t2Tensor, [A,C], [K,L]), Vertex(t2Tensor, [D,B], [I,J])]), TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [K,L], [C,D]), Vertex(t2Tensor, [B,C], [K,L]), Vertex(t2Tensor, [D,A], [J,I])])])

In [265]:
amplitudeDiagrams = [D1, D2a, D2b, D2c, D2d, D2e, D2ex1, D2ex2, D2ex3, D3a, D3b, D3bx1, D3bx2, D3bx3, D3bx4, D3bx5, D3c, D3cx, D3d, D3dx]
amplitudeFactors = [1., 1., -1., 0.5, 0.5, 2., -1., -1., -1., 0.5, 2., -2., 0.5, -1., 1., 0.5, -2., 1., -2., 1.]

In [207]:
for d, diagram in enumerate(amplitudeDiagrams):
    print(diagram)
    print(amplitudeFactors[d] * contractTensorSum(diagram))

1.0 * v_{AB}^{IJ}
[[[[0.18752198]]]]
1.0 * f_{B}^{C}{t_{2}}_{AC}^{IJ}
 + 1.0 * f_{A}^{C}{t_{2}}_{BC}^{JI}
[[[[0.14695891]]]]
1.0 * f_{K}^{J}{t_{2}}_{AB}^{IK}
 + 1.0 * f_{K}^{I}{t_{2}}_{BA}^{JK}
[[[[-0.32760552]]]]
1.0 * v_{AB}^{CD}{t_{2}}_{CD}^{IJ}
 + 1.0 * v_{BA}^{CD}{t_{2}}_{CD}^{JI}
[[[[-0.09335674]]]]
1.0 * v_{KL}^{IJ}{t_{2}}_{AB}^{KL}
 + 1.0 * v_{KL}^{JI}{t_{2}}_{BA}^{KL}
[[[[-0.0900353]]]]
1.0 * v_{KB}^{CJ}{t_{2}}_{AC}^{IK}
 + 1.0 * v_{KA}^{CI}{t_{2}}_{BC}^{JK}
[[[[-0.1032621]]]]
1.0 * v_{KB}^{CJ}{t_{2}}_{AC}^{KI}
 + 1.0 * v_{KA}^{CI}{t_{2}}_{BC}^{KJ}
[[[[0.05163105]]]]
1.0 * v_{KB}^{IC}{t_{2}}_{AC}^{KJ}
 + 1.0 * v_{KA}^{JC}{t_{2}}_{BC}^{KI}
[[[[0.17765866]]]]
1.0 * v_{KB}^{JC}{t_{2}}_{AC}^{IK}
 + 1.0 * v_{KA}^{IC}{t_{2}}_{BC}^{JK}
[[[[0.17765866]]]]
1.0 * v_{KL}^{CD}{t_{2}}_{CD}^{IJ}{t_{2}}_{AB}^{KL}
 + 1.0 * v_{KL}^{CD}{t_{2}}_{CD}^{JI}{t_{2}}_{BA}^{KL}
[[[[0.00355394]]]]
1.0 * v_{KL}^{CD}{t_{2}}_{AC}^{IK}{t_{2}}_{DB}^{LJ}
 + 1.0 * v_{KL}^{CD}{t_{2}}_{BC}^{JK}{t_{2}}_{DA}^{LI}


In [237]:
t2Tensor.array[0, 0, 0, 0] = -0.38

In [209]:
h2Tensor.array = get2bodyHamiltonianArray(mf)

In [266]:
residualTerms = [amplitudeFactors[d] * contractTensorSum(diagram) for d, diagram in enumerate(amplitudeDiagrams)]

In [267]:
residualTerms

[array([[[[0.18752198]]]]),
 array([[[[0.14695891]]]]),
 array([[[[-0.32760552]]]]),
 array([[[[-0.09335674]]]]),
 array([[[[-0.0900353]]]]),
 array([[[[-0.1032621]]]]),
 array([[[[0.05163105]]]]),
 array([[[[0.17765866]]]]),
 array([[[[0.17765866]]]]),
 array([[[[0.00355394]]]]),
 array([[[[0.01421575]]]]),
 array([[[[-0.01421575]]]]),
 array([[[[0.00355394]]]]),
 array([[[[-0.00710787]]]]),
 array([[[[0.00710787]]]]),
 array([[[[0.00355394]]]]),
 array([[[[-0.01421575]]]]),
 array([[[[0.00710787]]]]),
 array([[[[-0.01421575]]]]),
 array([[[[0.00710787]]]])]

In [323]:
constant = sum(residualTerms[:1])
linearFock = sum(residualTerms[1:3])
linearFluctuation = sum(residualTerms[3:9])
quadratic = sum(residualTerms[9:])

In [322]:
supposedLinearFluctuation = residualTerms[3] - residualTerms[4]
print(supposedLinearFluctuation)

[[[[-0.00332144]]]]


In [324]:
print(constant)
print(linearFock)
print(linearFluctuation)
print(quadratic)

[[[[0.18752198]]]]
[[[[-0.18064661]]]]
[[[[0.12029423]]]]
[[[[-0.00355394]]]]


In [327]:
constant + linearFock + supposedLinearFluctuation + quadratic

array([[[[1.6479873e-17]]]])

In [326]:
sum(residualTerms)

array([[[[0.12361567]]]])

In [262]:
sum(residualTerms[9:])

array([[[[-0.00355394]]]])

In [263]:
sum(residualTerms[:5]) + sum(residualTerms[9:])

array([[[[-0.18007061]]]])

In [308]:
Ej = TensorSum([TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [I,J], [A,B]), Vertex(t2Tensor, [A,B], [I,J])])])
Ek = TensorSum([TensorProduct([h2Diagrams[3], t2Tensor], vertexList=[Vertex(h2Diagrams[3], [I,J], [B,A]), Vertex(t2Tensor, [A,B], [I,J])])])

In [309]:
ECCD = 2 * contractTensorSum(Ej) - contractTensorSum(Ek)
print(ECCD)

-0.025815524316581975


In [257]:
t2Tensor.array * ECCD

array([[[[0.00355394]]]])

In [303]:
print(Ej)
print(Ek)

1.0 * v_{IJ}^{AB}{t_{2}}_{AB}^{IJ}
1.0 * v_{IJ}^{BA}{t_{2}}_{AB}^{IJ}


In [349]:
from importlib import reload

In [350]:
import texify
#reload(texify)

<module 'texify' from '/Users/alexgunasekera/Documents/DPhil/code/DPhil/COSCC/texify.py'>

In [348]:

texify.texify([Ej, Ek], "CCDEnergyTerms")